# Neural Network Recommendation System

### Load the data set

In [1]:
# Tensorflow and tensorflow-recommenders needs to be installed for this recommendation system
# %pip install tensorflow tensorflow-recommenders

In [2]:
# Import necessary modules
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-10-25 15:57:14.054424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Import the data
recipe_df = pd.read_csv('../Data/Cleaned/recipes_df_cleaned.csv')
interactions_df = pd.read_csv('../Data/Cleaned/interactions_df_cleaned.csv')

In [4]:
recipe_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [97]:
recipe_df['id'].nunique()

231637

In [5]:
interactions_df.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


### Preparing the Model - Data Preprocessing

I need to create a model that will embed users and recipes into a high-dimensional vecor space.  In tensorflow this is called embedding.

##### Interactions Dataframe

In [6]:
# # I'm going to drop the date column as it is not needed for our recommendation system
# interactions_df = interactions_df.drop(columns = 'date')

In [7]:
# # Next I'm going to tokenize the review column.  This converts the strings into numerical values
# interactions_df['review'] = interactions_df['review'].astype(str)
# review_tokenizer = Tokenizer()
# review_tokenizer.fit_on_texts(interactions_df['review'])
# review_sequences = review_tokenizer.texts_to_sequences(interactions_df['review'])

In [8]:
# # Pad the review_sequences to ensure consistency of sequence length
# max_review_length = max(len(seq) for seq in review_sequences)
# review_padded = pad_sequences(review_sequences, maxlen=max_review_length, padding='post')

In [9]:
# # Convert the dataframes into a tensorflow dataset.
# # This isn't necessary but will allow the model to be more efficient and make full use of all of tensorflow's capabilities

# features = {
#     'user_id': interactions_df['user_id'].values,
#     'recipe_id': interactions_df['recipe_id'].values,
#     'review': review_padded
# }

# labels = interactions_df['rating']

# interactions_dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [10]:
# # Save the tensorflow dataset so i don't have to recreate it every time
# interactions_dataset.save('../Data/Preprocessed/TensorFlow_Dataset')


##### Recipe Dataframe

In [11]:
# # Drop the columns we will not use:
# recipe_df = recipe_df.drop(columns = 'submitted')

In [12]:
# # Now lets convert the tags, ingredients, and steps columns into strings

# # The tags and ingredients column is a string representation of a list. This converts it to an actual list
# recipe_df['tags'] = recipe_df['tags'].apply(lambda x: ast.literal_eval(x))
# recipe_df['ingredients'] = recipe_df['ingredients'].apply(lambda x: ast.literal_eval(x))
# recipe_df['steps'] = recipe_df['steps'].apply(lambda x: ast.literal_eval(x))

# # Convert the lists to strings:
# recipe_df['tags'] = recipe_df['tags'].apply(lambda x: ' '.join(x))
# recipe_df['ingredients'] = recipe_df['ingredients'].apply(lambda x: ' '.join(x))
# recipe_df['steps'] = recipe_df['steps'].apply(lambda x: ' '.join(x))

In [13]:
# # Now I need to convert the nutrition column into floats and separate the listed values into the appropriate columns

# # Convert the nutrition column from string representation to an actual list of floats:
# recipe_df['nutrition'] = recipe_df['nutrition'].apply(lambda x: ast.literal_eval(x))

# # Expand the nutrition information into separate columns
# # Per the dataset documentation, the columns are ['calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']
# nutrition_df = pd.DataFrame(recipe_df['nutrition'].tolist(), columns = ['calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates'])

In [14]:
# # Concatenate the nutritional information with the recipe_df
# recipe_df = pd.concat([recipe_df, nutrition_df], axis = 1)

# # Drop the original nutrition column, since that information is now contained in separate columns
# recipe_df.drop(columns = ['nutrition'], inplace = True)

In [15]:
# from tqdm import tqdm

# # Tokenize and pad the columns which are strings

# # The following columns need to be tokenized: ['name', 'tags', 'steps', 'description', 'ingredients']
# columns_to_tokenize = ['name', 'tags', 'steps', 'description', 'ingredients']

# padded_sequences = {}
# for column in tqdm(columns_to_tokenize):
#     recipe_df[column] = recipe_df[column].astype(str)
#     tokenizer = Tokenizer()
#     tokenizer.fit_on_texts(recipe_df[column])
#     sequences = tokenizer.texts_to_sequences(recipe_df[column])
#     max_length = max(len(seq) for seq in sequences)
#     seq_padded = pad_sequences(sequences, maxlen=max_length, padding='post')
#     padded_sequences[column+'_seq'] = seq_padded

In [16]:
# # Convert the dataframes into a tensorflow dataset.
# # This isn't necessary but will allow the model to be more efficient and make full use of all of tensorflow's capabilities

# features = {
#     'name': padded_sequences['name_seq'],
#     'recipe_id': recipe_df['id'].values,
#     'minutes': recipe_df['minutes'].values,
#     'contributor_id': recipe_df['contributor_id'].values,
#     'tags': padded_sequences['tags_seq'],
#     'n_steps': recipe_df['n_steps'].values,
#     'steps': padded_sequences['steps_seq'],
#     'description': padded_sequences['description_seq'],
#     'ingredients': padded_sequences['ingredients_seq'],
#     'n_ingredients': recipe_df['n_ingredients'].values,
#     'calories': recipe_df['calories'].values,
#     'total_fat': recipe_df['total_fat'].values,
#     'sugar': recipe_df['sugar'].values,
#     'sodium': recipe_df['sodium'].values,
#     'protein': recipe_df['protein'].values,
#     'saturated_fat': recipe_df['saturated_fat'].values,
#     'carbohydrates': recipe_df['carbohydrates'].values
# }


# recipe_dataset = tf.data.Dataset.from_tensor_slices(features)

In [17]:
# # Save the tensorflow dataset so i don't have to recreate it every time
# recipe_dataset.save('../Data/Preprocessed/Recipe_TensorFlow_Dataset')

In [18]:
# Load the preprocessed tensorflow datasets:
interactions_dataset = tf.data.Dataset.load('../Data/Preprocessed/Interactions_TensorFlow_Dataset')
recipe_dataset = tf.data.Dataset.load('../Data/Preprocessed/Recipe_TensorFlow_Dataset')

### Build the retrieval model

A retrieval model acts as the first stage in a recommendation system, filtering a vast array of options to present a user with a curated list of items that are most likely to meet their needs and interests. Our model will use the user_id and recipe_id to identify which recipes the users has made and retrieve a short list of recipes that are similar to the ones already made.

In [19]:
# For this retrieval model I'll only use the recipe_id and the user_id
ratings = interactions_dataset.map(lambda x,y: {
    'recipe_id': x['recipe_id'],
    'user_id': x['user_id']
})

recipes = recipe_dataset.map(lambda x: x['recipe_id'])

In [20]:
# Let's split the dataset into training and test sets
tf.random.set_seed(123)
train_set_length = int(round(len(interactions_df)*0.7,0))
test_set_length = int(round(len(interactions_df)*0.3,0))
shuffled = ratings.shuffle(len(interactions_df), seed = 123, reshuffle_each_iteration=False)
train = shuffled.take(train_set_length)
test = shuffled.skip(train_set_length).take(test_set_length)

In [21]:
# To execute this model I need to determine what the unique recipe ids and unique user ids are
# These unique ids will be used as the "vocabulary" and embedded and within their respective models
recipe_ids = recipes.batch(1_000)
user_ids = ratings.batch(10_000).map(lambda x: x['user_id'])

unique_recipes = np.unique(np.concatenate(list(recipe_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [22]:
# Create representation models for user_id and recipe_ids:
embedding_dimension = 32

user_lookup_layer = tf.keras.layers.IntegerLookup()
user_lookup_layer.adapt(unique_user_ids)

recipe_lookup_layer = tf.keras.layers.IntegerLookup()
recipe_lookup_layer.adapt(unique_recipes)

user_model = tf.keras.Sequential([
    user_lookup_layer,
    tf.keras.layers.Embedding(input_dim = len(unique_user_ids)+1, output_dim = embedding_dimension)
])

recipe_model = tf.keras.Sequential([
    recipe_lookup_layer,
    tf.keras.layers.Embedding(input_dim = len(unique_recipes)+1, output_dim = embedding_dimension)
])

In [23]:
# Establish the metrics to be used to evaluate the model
metrics = tfrs.metrics.FactorizedTopK(
    candidates= recipes.batch(100).map(recipe_model)
)

The retrieval function defines a model that will retrieve recipes from the recipe dataset by minimizing crossentropy loss. This loss function works by comparing a user's embedding with both the positive (correct) recipe and a set of negative (random candidate) recipes. The model learns to make the positive recipe closer to the user in embedding space and push the negative recipes further away.

During training, the model tries to maximize the dot product between the user embedding and the correct recipe embedding (i.e., the one the user interacted with). Simultaneously, it minimizes the dot product with negative samples (random recipes that the user hasn’t interacted with).

In [24]:
# Establish the task using the retrieval function

task = tfrs.tasks.Retrieval(
    metrics=metrics
)

In [25]:
class RecipeRetrievalModel(tfrs.Model):

    def __init__(self, user_model, recipe_model):
        super().__init__()
        self.recipe_model: tf.keras.Model = recipe_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: dict[str, tf.Tensor], training = False) -> tf.Tensor:
        # Pick out user features and pass them into the user model
        user_embeddings = self.user_model(features['user_id'])
        # Pick out the recipe features and pass them into the recipe model
        recipe_embeddings = self.recipe_model(features['recipe_id'])
        # Compute the loss and metrics
        return self.task(user_embeddings, recipe_embeddings, compute_metrics = not training)

    def call(self, features: dict[str, tf.Tensor]) -> tf.Tensor:
        # This will define the model’s forward pass for saving
        user_embeddings = self.user_model(features['user_id'])
        recipe_embeddings = self.recipe_model(features['recipe_id'])
        return user_embeddings, recipe_embeddings

In [26]:
model = RecipeRetrievalModel(user_model, recipe_model)
model.compile(optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [27]:
# Establish the training and the test data sets
cached_train = train.shuffle(1000000).batch(10000).cache()
cached_test = test.batch(1000).cache()

In [28]:
history = model.fit(cached_train, epochs = 5, verbose=1)

Epoch 1/5
80/80 [==============================] - 242s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 89741.8414 - regularization_loss: 0.0000e+00 - total_loss: 89741.8414
Epoch 2/5
80/80 [==============================] - 112s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 87111.3001 - regularization_loss: 0.0000e+00 - total_loss: 87111.3001
Epoch 3/5
80/80 [==============================] - 110s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - fact

In [29]:
# # Evaluate the retrieval model
# retrieval_model_eval_metrics = model.evaluate(cached_test, return_dict = True, verbose = 1)

#### Pull a set of recipe recommendations from the retrieval model

In [30]:
# Create a model that will take in user ids as inputs and outputs a list of 100 recipes from the entire recipe dataset

# The code below calculates the similarity between user and recipe embeddings by comparing every possible user-recipe pair (brute force).
recipe_index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

# The code below sets up a searchable index of recipes based on their embeddings.
recipe_index.index_from_dataset(
    tf.data.Dataset.zip((recipes.batch(100), recipes.batch(100).map(model.recipe_model))) # Converts each batch of recipe IDs into recipe embeddings by passing them through model.recipe_model.
)

2024-10-25 16:14:21.296638: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype resource
	 [[{{node Placeholder/_14}}]]
2024-10-25 16:14:21.297250: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype resource
	 [[{{node Placeholder/_14}}]]


In [147]:
len(interactions_df[interactions_df['user_id']==92385])

301

In [148]:
# Define a function that will input the userid and produce a list of recommended recipe names from the retrieval model
def Retrieval_Recommendations_user(user_id, num_results = 30):
    '''
    Input a user id and returns a list of recipe names and ids that are likely to be of interest
    to the user based on their previous interactions. This filters out recipes which the user has already
    made.
    '''    
    num_interactions = len(interactions_df[interactions_df['user_id']==user_id])
    num_results = num_interactions + 30
    
    _, recipe_id = recipe_index(tf.constant([user_id]), k = num_results)
    
    # Convert recipe_id to a list of recipe IDs
    recipe_id = recipe_id.numpy().flatten().tolist()
    
    # Filter out recipes which the user has already cooked
    cooked_recipes = interactions_df[interactions_df['user_id'] == user_id]['recipe_id'].values
    recommended_recipe_ids = [id for id in recipe_id if id not in cooked_recipes]

    # Produce recipe names from the recipe ids
    recommended_recipe_names = recipe_df[recipe_df['id'].isin(recommended_recipe_ids)]['name'].tolist()
    
    return recommended_recipe_names, recommended_recipe_ids

In [51]:
# Test the retrieval recommendations function
Retrieval_Recommendations_user(38094)

(['slightly spicy  black bean burgers',
  'balsamic chicken thighs',
  'brussels sprouts in garlic butter',
  'burritos for the crock pot',
  'chicken breasts with spicy honey orange glaze',
  'chicken fried rice   oamc',
  'chickpeas with spinach  greek',
  'chinese beef and broccoli',
  'finger lickin good bbq sauce',
  'ground beef with cabbage',
  'gumbo',
  'mary s blue cheese dressing',
  'mocha mousse  with tofu',
  'puppy chow snack mix',
  'quaker oats meatloaf',
  'spicy croutons',
  'spicy smoky soulful lentil soup',
  'spinach   cheese grilled sandwich',
  'thai peanut stir fry sauce',
  'tomato basil soup',
  'whole wheat cornbread'],
 [76250,
  21561,
  32042,
  80963,
  81968,
  48663,
  38444,
  1267,
  110139,
  111462,
  5297,
  19023,
  40654,
  28249,
  1035,
  71557,
  8654,
  21007,
  70362,
  21761,
  86350])

#### Save the retrieval model

In [33]:
# Save the model
# tf.keras.models.save_model(recipe_index, '../Models/model_retrieval1_bf')

In [34]:
# Save the training history and eval metrics

# retrieval_model_training_metrics = history.history
# history_df = pd.DataFrame(retrieval_model_training_metrics)
# history_df.to_csv('../Models/Model_Metrics/model_retrieval1_history.csv')
# history_df.head()

# eval_df = pd.DataFrame([retrieval_model_eval_metrics])
# eval_df.to_csv('../Models/Model_Metrics/model_retrieval1_eval.csv')
# eval_df.head()

### Build the ranking model

The ranking model takes the outputs of the retrieval model and selects the best possible handful of recommendations based on other features in the interactions dataset such as ratings. Its task is to narrow down the set of items the user may be interested in to a shortlist of likely candidates.

#### Load the retrieval model

In [35]:
# retrieval_model = tf.keras.models.load_model('../Models/model_retrieval1_bf')

#### Prepare the dataset

For this model I'll use the interactions dataset and include the previous recipe rankings by the user

In [36]:
ratings = interactions_dataset.map(lambda x, y:{
    'recipe_id': x['recipe_id'],
    'user_id': x['user_id'],
    'rating': y
})

In [37]:
# Split the dataset into training and test sets
tf.random.set_seed(123)
train_set_length = int(round(len(interactions_df)*0.7,0))
test_set_length = int(round(len(interactions_df)*0.3,0))
shuffled = ratings.shuffle(len(interactions_df), seed = 123, reshuffle_each_iteration=False)
train = shuffled.take(train_set_length)
test = shuffled.skip(train_set_length).take(test_set_length)

#### Build the rankings model

First, I'll create a model architecture that will take a user id and recipe id as inputs and return a predicted rating using the previously trained user and recipe embeddings

In [38]:
class RankingModel(tf.keras.Model):

    def __init__(self, user_model, recipe_model):
        super().__init__()
        
        # Import the user embeddings from the retrieval model. This incorporates the retrieval model learnings
        # i.e., the embeddings contain information related to which recipes the user is likely to choose based on which ones they've already interacted with
        self.user_model: tf.keras.Model = user_model
        self.recipe_model: tf.keras.Model = recipe_model

        # Build a sequential model to predict the rankings
        self.rating_model: tf.keras.Sequential = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(256, activation = 'relu'),
                tf.keras.layers.Dense(64, activation = 'relu'),
                tf.keras.layers.Dense(1) # This will provide a continuous output to predict the rating of the recipe for the particular user
            ])
        

    def call(self, inputs):
        user_id, recipe_id = inputs #This model will take a user_id and recipe_id as inputs
        user_embeddings = self.user_model(user_id)
        recipe_embeddings = self.recipe_model(recipe_id)
        concatenated = tf.concat([user_embeddings, recipe_embeddings], axis=1)
        return self.rating_model(concatenated) # The model will output a rating of the recipe for the particular user
        

        

In [39]:
# Test the ranking model.  It should produce a predicted rating when fed a user id and recipe id
RankingModel(user_model, recipe_model)((tf.constant([38094]),tf.constant([40753])))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.10222232]], dtype=float32)>

In [40]:
# Define the task of the recipe ranking model, which will be to minimize the MSE of predicted ratings

task = tfrs.tasks.Ranking(
    loss = tf.keras.losses.MeanSquaredError(), # We'll use MSE as the loss function for the ranking model
    metrics = [tf.keras.metrics.RootMeanSquaredError()] # We'll use RMSE as the metric to evaluate the ranking model
)


In [41]:
class RecipeRatingModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel(user_model, recipe_model)
        self.task: tf.keras.layers.Layer = task

    def call(self, features: dict[str, tf.Tensor]) -> tf.Tensor:
        return self.ranking_model(
            (features['user_id'], features['recipe_id']))

    def compute_loss(self, features: dict[str, tf.Tensor], training = False) -> tf.Tensor:
        labels = features.pop('rating')

        predicted_ratings = self(features)
    
        loss = self.task(labels = labels, predictions = predicted_ratings)
        
        return loss

In [42]:
# Establish the training and the test data sets
cached_train = train.shuffle(1000000).batch(10000).cache()
cached_test = test.batch(1000).cache()

In [43]:
recipe_rating_model = RecipeRatingModel()
recipe_rating_model.compile(optimizer = tf.keras.optimizers.Adagrad(learning_rate = 0.1))
history = recipe_rating_model.fit(cached_train, epochs = 5, verbose = 1)

Epoch 1/5
80/80 [==============================] - 127s 102ms/step - root_mean_squared_error: 1.9464 - loss: 3.7407 - regularization_loss: 0.0000e+00 - total_loss: 3.7407
Epoch 2/5
80/80 [==============================] - 4s 48ms/step - root_mean_squared_error: 1.2376 - loss: 1.5317 - regularization_loss: 0.0000e+00 - total_loss: 1.5317
Epoch 3/5
80/80 [==============================] - 4s 45ms/step - root_mean_squared_error: 1.2344 - loss: 1.5239 - regularization_loss: 0.0000e+00 - total_loss: 1.5239
Epoch 4/5
80/80 [==============================] - 4s 48ms/step - root_mean_squared_error: 1.2306 - loss: 1.5145 - regularization_loss: 0.0000e+00 - total_loss: 1.5145
Epoch 5/5
80/80 [==============================] - 4s 48ms/step - root_mean_squared_error: 1.2290 - loss: 1.5106 - regularization_loss: 0.0000e+00 - total_loss: 1.5106


In [44]:
recipe_rating_model.evaluate(cached_test, return_dict = True, verbose = 1)

340/340 [==============================] - 119s 11ms/step - root_mean_squared_error: 1.2278 - loss: 1.5069 - regularization_loss: 0.0000e+00 - total_loss: 1.5069


{'root_mean_squared_error': 1.2278293371200562,
 'loss': 1.3361842632293701,
 'regularization_loss': 0,
 'total_loss': 1.3361842632293701}

In [110]:
# Let's test the ranking model to ensure it is working correctly:

test_ratings = {}
test_recipe_ids = interactions_df['recipe_id'].sample(n=10).tolist()
for id in test_recipe_ids:
    test_ratings[id] = recipe_rating_model({
        'user_id': np.array([38094]),
        'recipe_id': np.array([id])
    })

print('Ratings:')
for id, score in sorted(test_ratings.items(), key = lambda x: x[1], reverse = True):
    print(f'{id}:{score}')


Ratings:
112624:[[4.49862]]
493413:[[4.334894]]
292963:[[4.302121]]
233830:[[4.262704]]
429438:[[4.2279067]]
157144:[[4.175563]]
447553:[[4.1562414]]
146058:[[4.132432]]
129368:[[4.111205]]
1356:[[4.099944]]


In [115]:
# Create a function that will take a user id and list of recipes as inputs and generate predicted rankings for the recipes for that specific user.
def Ranking_Recommendations_user(user_id, recipe_ids):
    
    '''
    Input a user id and list of recipe ids and the function will use the ranking model to return
    a list of 5 recipes the user should try next.
    '''
    
    predicted_ratings = {}
    for recipe_id in recipe_ids:
        predicted_ratings[recipe_id] = recipe_rating_model({
            'user_id': np.array([user_id]),
            'recipe_id': np.array([recipe_id])
        })
    
    sorted_ratings = sorted(predicted_ratings.items(), key = lambda x: x[1], reverse = True)

    recipe_recommendation_names = {}
    final_recipe_recommendations = []
    
    for id, score in sorted_ratings:
        recipe_recommendation_names[id] = recipe_df[recipe_df['id']== id]['name'].iloc[0]

    for i, (id,name) in enumerate(recipe_recommendation_names.items()):
        if i < 5:
            final_recipe_recommendations.append(name)

    return final_recipe_recommendations
        

In [117]:
Ranking_Recommendations_user(38094, test_recipe_ids)

['peanutty sesame noodles',
 'loaded potato and buffalo chicken casserole',
 'bek s grilled mahi mahi fillets in soy ginger marinade  oamc',
 'apricot carrots',
 'mcdonald s big mac sauce copycat recipe']

In [45]:
# Save the training history and eval metrics

ranking_model_training_metrics = history.history
history_df = pd.DataFrame(ranking_model_training_metrics)
history_df.to_csv('../Models/Model_Metrics/model_ranking1_history.csv')
history_df.head()

eval_df = pd.DataFrame([Out[44]])
eval_df.to_csv('../Models/Model_Metrics/model_ranking1_eval.csv')
eval_df.head()

,root_mean_squared_error,loss,regularization_loss,total_loss
0,1.227829,1.336184,0,1.336184


## Make Recommendations

I will use the Retrieval_Recommendations_user function built above to create a list of 30 recipes that are most likely to be of interest to the user based on their previous interactions. This list of 30 recipes will be used as an input into the Ranking Recommendations model to predict ratings for the specific user.  The function selects the top 5 predicted ratings to recommend to the user and prints them to the console.

In [163]:
def RecipeRecommendationFull(user_id):
    retrieval_rec_names, retrieval_rec_ids = Retrieval_Recommendations_user(user_id)
    recipe_recs = Ranking_Recommendations_user(user_id, retrieval_rec_ids)
    return recipe_recs

In [166]:
# Select a  user:
random_user = interactions_df['user_id'].sample(n=1).iloc[0]

# Obtain a list of 30 recipes the user is likely to prefer from the retreival neural network model and use that list
# as an input into the ranking model. Return a list of 5 recipes which had the highest predicted rating
recipe_recommendations = RecipeRecommendationFull(random_user)

# Print the recommended recipes to the console
print(f'Recipe recommendations for user {random_user}:')
for recipe in recipe_recommendations:
    print('\t' * 1 + recipe)

Recipe recommendations for user 2001024304:
	perfect pork tenderloin
	pete s scratch pancakes
	p  f  chang s mongolian beef by todd wilbur
	best banana bread
	breakfast shepherd s pie
